In [111]:
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
import pandas as pd
import opensmile

In [112]:
df = pd.DataFrame({ 'id': [fn[:3] for fn in os.listdir('./dataset') if fn.endswith('.wav')] })
df.head()

,id
0,300
1,301
2,302
3,303
4,304


In [113]:
is_depressed = {}
for fn in [
    'dev_split_Depression_AVEC2017.csv',
    'train_split_Depression_AVEC2017.csv',
    'test_split_Depression_AVEC2017.csv',
    'full_test_split.csv']:
    with open('./dataset/'+fn, 'r') as f:
        for row in f.readlines()[1:]:
            vals = row[:-1].split(',')
            if len(vals) >= 2:
                is_depressed[vals[0]] = int(vals[1])
df['is_depressed'] = df['id'].apply(lambda id: is_depressed[id] if id in is_depressed else 0)
df.head()

,id,is_depressed
0,300,0
1,301,0
2,302,0
3,303,0
4,304,0


In [114]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)
smile_df = smile.process_files([f'./dataset/{id}_AUDIO.wav' for id in df['id'].tolist()])
smile_df.head()

,,,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
file,start,end,,,,,,,,,,,,,,,,,,,,,
./dataset/300_AUDIO.wav,0 days,0 days 00:10:48.500000,2.835027,0.331277,0.016178,0.073676,0.092670,0.169379,0.018994,0.076709,0.095703,0.060193,...,8.667942,0.601993,1.848118,1.848170,-20.0,0.581762,89.249535,44.508762,88.691612,44.451393
./dataset/301_AUDIO.wav,0 days,0 days 00:13:43.900000,1.563116,0.992086,0.023573,0.065601,0.094496,0.226772,0.028895,0.132276,0.161170,0.059034,...,9.814976,0.585230,2.147391,2.147398,-20.0,0.601254,95.877174,49.022385,94.419037,48.620708
./dataset/302_AUDIO.wav,0 days,0 days 00:12:38.800000,0.844108,0.954160,0.000699,0.061128,0.065454,0.101554,0.004327,0.036100,0.040426,0.056394,...,9.711348,0.589838,1.997163,1.997292,-20.0,0.609909,93.560135,47.109814,93.347351,46.971210
./dataset/303_AUDIO.wav,0 days,0 days 00:16:25.300000,1.446217,0.840322,0.039026,0.062715,0.098875,0.221331,0.036160,0.122456,0.158616,0.054199,...,11.710479,0.557672,2.333878,2.333952,-20.0,0.636285,97.992241,51.536892,97.227127,51.981255
./dataset/304_AUDIO.wav,0 days,0 days 00:13:12.600000,1.083328,0.410899,0.023747,0.057617,0.074131,0.142586,0.016514,0.068455,0.084969,0.051853,...,10.999923,0.598984,2.158344,2.158308,20.0,0.548527,97.093452,48.878418,95.413612,50.013947


In [147]:
#smile_df = smile_df.drop('file', axis=1)
X_train, X_test, y_train, y_test = train_test_split(smile_df, df['is_depressed'], stratify=df['is_depressed'])

inp = layers.Input(shape=(6373))
out = layers.Dense(128, activation="relu")(inp)
out = layers.Dense(64, activation="relu")(out)
out = layers.Dense(32, activation="relu")(out)
out = layers.Dense(1, activation='sigmoid')(out)
model = Model(inputs=inp, outputs=out)

optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')
model.summary()

ValueError: Input 0 of layer "bidirectional_10" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 96)

In [116]:
model.fit(x=X_train, y=y_train, epochs=50)

Epoch 1/50
5/5 [==============================] - 1s 9ms/step - loss: 205852768.0000
Epoch 2/50
5/5 [==============================] - 0s 9ms/step - loss: 83460744.0000
Epoch 3/50
5/5 [==============================] - 0s 9ms/step - loss: 43654880.0000
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 31740808.0000
Epoch 5/50
5/5 [==============================] - 0s 9ms/step - loss: 22134702.0000
Epoch 6/50
5/5 [==============================] - 0s 9ms/step - loss: 12321625.0000
Epoch 7/50
5/5 [==============================] - 0s 9ms/step - loss: 25171316.0000
Epoch 8/50
5/5 [==============================] - 0s 9ms/step - loss: 30381056.0000
Epoch 9/50
5/5 [==============================] - 0s 9ms/step - loss: 35615936.0000
Epoch 10/50
5/5 [==============================] - 0s 9ms/step - loss: 21044210.0000
Epoch 11/50
5/5 [==============================] - 0s 9ms/step - loss: 28782388.0000
Epoch 12/50
5/5 [==============================] - 0s 9ms/step - loss: 19

In [117]:
pred = model.predict(X_test)
print(classification_report(y_test, pred))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        32
           1       0.00      0.00      0.00        14

    accuracy                           0.70        46
   macro avg       0.35      0.50      0.41        46
weighted avg       0.48      0.70      0.57        46



C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [118]:
def get_text(id):
    lines = []
    with open(f'./dataset/{id}_TRANSCRIPT.csv', 'r') as f:
        for line in f.readlines()[1:]:
            try:
                if line.split('\t')[2] == 'Participant':
                    lines.append(line.replace('\n', '').split('\t')[-1])
            except Exception:
                pass
    return '\n'.join(lines)

bert_df['text'] = df['id'].apply(lambda id: get_text(id))
bert_df.head()

0    good\natlanta georgia\num my parents are from ...
1    thank you\nmmm k\ni'm doing good thank you\ni'...
2    i'm fine how about yourself \ni'm from los ang...
3    okay how 'bout yourself\nhere in california\ny...
4    i'm doing good um\nfrom los angeles california...
Name: id, dtype: object

In [119]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(bert_df['text'])
sequences = tokenizer.texts_to_sequences(bert_df['text'])

max_length = max(len(x) for x in sequences)
bert_df['text'] = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length)

In [120]:
tX_train, tX_test, ty_train, ty_test = train_test_split(bert_df['text'], df['is_depressed'], stratify=df['is_depressed'])

tinp = layers.Input(shape=(4618))
tout = layers.Dense(128, activation="relu")(tinp)
tout = layers.Dense(64, activation="relu")(tout)
tout = layers.Dense(32, activation="relu")(tout)
tout = layers.Dense(1, activation='sigmoid')(tout)
tmodel = Model(inputs=tinp, outputs=tout)

tmodel.compile(optimizer='adam', loss='binary_crossentropy')
tmodel.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 4618)]            0         
                                                                 
 dense_55 (Dense)            (None, 128)               591232    
                                                                 
 dense_56 (Dense)            (None, 64)                8256      
                                                                 
 dense_57 (Dense)            (None, 32)                2080      
                                                                 
 dense_58 (Dense)            (None, 1)                 33        
                                                                 
Total params: 601601 (2.29 MB)
Trainable params: 601601 (2.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [121]:
tmodel.fit(x=tX_train, y=ty_train, epochs=20)

Epoch 1/20
5/5 [==============================] - 1s 7ms/step - loss: 106.7402
Epoch 2/20
5/5 [==============================] - 0s 7ms/step - loss: 18.0551
Epoch 3/20
5/5 [==============================] - 0s 7ms/step - loss: 4.6407
Epoch 4/20
5/5 [==============================] - 0s 8ms/step - loss: 0.9206
Epoch 5/20
5/5 [==============================] - 0s 7ms/step - loss: 1.6184
Epoch 6/20
5/5 [==============================] - 0s 7ms/step - loss: 1.5703
Epoch 7/20
5/5 [==============================] - 0s 7ms/step - loss: 0.3864
Epoch 8/20
5/5 [==============================] - 0s 7ms/step - loss: 0.8782
Epoch 9/20
5/5 [==============================] - 0s 7ms/step - loss: 6.2456e-11
Epoch 10/20
5/5 [==============================] - 0s 7ms/step - loss: 4.2400e-09
Epoch 11/20
5/5 [==============================] - 0s 7ms/step - loss: 0.0553
Epoch 12/20
5/5 [==============================] - 0s 7ms/step - loss: 4.1356e-12
Epoch 13/20
5/5 [==============================] - 0s 7ms/

In [122]:
tpred = tmodel.predict(tX_test)
print(classification_report(ty_test, tpred.round()))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.68      0.88      0.77        32
           1       0.20      0.07      0.11        14

    accuracy                           0.63        46
   macro avg       0.44      0.47      0.44        46
weighted avg       0.54      0.63      0.57        46



In [145]:
combined = layers.concatenate([model.output, tmodel.output])
ttout = layers.Dense(128, activation="relu")(combined)
ttout = layers.Dense(128, activation="relu")(ttout)
ttout = layers.Dense(64, activation="relu")(ttout)
ttout = layers.Dense(32, activation="relu")(ttout)
ttout = layers.Dense(1, activation='sigmoid')(ttout)
ttmodel = Model(inputs=[model.input, tmodel.input], outputs=ttout)

ttmodel.compile(optimizer='adam', loss='binary_crossentropy')
ttmodel.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_21 (InputLayer)       [(None, 6373)]               0         []                            
                                                                                                  
 input_22 (InputLayer)       [(None, 4618)]               0         []                            
                                                                                                  
 dense_51 (Dense)            (None, 128)                  815872    ['input_21[0][0]']            
                                                                                                  
 dense_55 (Dense)            (None, 128)                  591232    ['input_22[0][0]']            
                                                                                           

In [137]:
ttmodel.fit(x=[X_train, tX_train], y=ty_train, epochs=50)

Epoch 1/50
5/5 [==============================] - 2s 16ms/step - loss: 0.6846
Epoch 2/50
5/5 [==============================] - 0s 16ms/step - loss: 0.6547
Epoch 3/50
5/5 [==============================] - 0s 16ms/step - loss: 0.6250
Epoch 4/50
5/5 [==============================] - 0s 16ms/step - loss: 0.6228
Epoch 5/50
5/5 [==============================] - 0s 18ms/step - loss: 0.6180
Epoch 6/50
5/5 [==============================] - 0s 22ms/step - loss: 0.6176
Epoch 7/50
5/5 [==============================] - 0s 17ms/step - loss: 0.6165
Epoch 8/50
5/5 [==============================] - 0s 16ms/step - loss: 0.6169
Epoch 9/50
5/5 [==============================] - 0s 16ms/step - loss: 0.6164
Epoch 10/50
5/5 [==============================] - 0s 16ms/step - loss: 0.6165
Epoch 11/50
5/5 [==============================] - 0s 16ms/step - loss: 0.6173
Epoch 12/50
5/5 [==============================] - 0s 17ms/step - loss: 0.6169
Epoch 13/50
5/5 [==============================] - 0s 17ms/st

In [146]:
ttpred = ttmodel.predict([X_test, tX_test])
print(classification_report(ty_test, ttpred.round()))

2/2 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        32
           1       0.00      0.00      0.00        14

    accuracy                           0.70        46
   macro avg       0.35      0.50      0.41        46
weighted avg       0.48      0.70      0.57        46



C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)